In [ ]:
from dotenv import load_dotenv
load_dotenv()

- Change the parameter in _book_time_entry_ to __projectId__
- Add a new tool that gets a project list

In [ ]:
from langchain_core.tools import tool

# Tools

@tool
def get_projects():
    """
    Returns a list of mock time entry projects for a software consulting company.
    Each project includes client name, project name, and project ID.
    """
    return [
        {
            "client_name": "Acme Corporation",
            "project_name": "E-commerce Platform Redesign",
            "project_id": "aef12d88-e0ae-4949-b7a2-7704f3b0c1d0"
        },
        {
            "client_name": "TechNova Inc.",
            "project_name": "Mobile App Development",
            "project_id": "b57cf8e2-3714-4b83-9d63-afcd23e0f87b"
        },
        {
            "client_name": "Global Finance Group",
            "project_name": "Payment Processing System",
            "project_id": "c982b427-59a5-4c5e-b3f6-76ae4c1d9e5d"
        },
        {
            "client_name": "HealthCare Solutions",
            "project_name": "Patient Management Portal",
            "project_id": "d341f57a-6890-4f13-a8de-92c23481b6c2"
        },
        {
            "client_name": "EduTech Ventures",
            "project_name": "Learning Management System",
            "project_id": "e725d1b9-4a07-41b8-9c15-3f7b60c84a36"
        },
        {
            "client_name": "Intertech",
            "project_name": "Paid Time Off",
            "project_id": "f109e384-5c27-49f5-b651-48e72a0dca7f"
        }
    ]

@tool
def book_time_entry(projectId: str, date, hours: int = 8):
    """
    Book time entry for a project.
    Args:
        projectId (str): The unique ID of the project.
        date (str): The date for the time entry.
        hours (int): The number of hours to book.
    """
    return f"Booked {hours} hours on {date} for {projectId}."

tools = [get_projects, book_time_entry]

Define a langauage model and bind the tools.

In [ ]:
from langchain_anthropic import ChatAnthropic

# Language Model
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")
llm = llm.bind_tools(tools)

Build our graph.

In [ ]:
from datetime import date
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition

# Graph
builder = StateGraph(MessagesState)

# System message
sys_msg = SystemMessage(content=f"""
    You are a helpful assistant tasked with helping Intertech employees track time spent on projects.
    Today is {date.today().isoformat()}.
    """)

# Nodes

def assistant(state: MessagesState):
    return {"messages": [llm.invoke([sys_msg] + state["messages"])]}

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

# Build the graph
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

Display the graph.

In [ ]:

from IPython.display import Image, display

# Show
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

messages = [HumanMessage(content="I took PTO on Monday.")]
messages = graph.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()